In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/titanic/train.csv")
test = pd.read_csv("/kaggle/input/titanic/test.csv")

In [ ]:
train.shape, test.shape

In [ ]:
train.head()

In [ ]:
train.info(), test.info()

In [ ]:
train.drop(columns=["Cabin"], inplace = True)
test.drop(columns = ["Cabin"], inplace = True)

In [ ]:
train.isnull().sum(), test.isnull().sum()

## Handling Missing Values

In [ ]:
train["Embarked"].value_counts()

In [ ]:
train["Embarked"].fillna("S", inplace = True)

In [ ]:
train.isnull().sum()

In [ ]:
test["Fare"].fillna(test["Fare"].mean(), inplace = True)

In [ ]:
test.isnull().sum()

In [ ]:
male_age_train = np.random.randint(train[train["Sex"]=="male"]["Age"].mean()-train[train["Sex"]=="male"]["Age"].std(),train[train["Sex"]=="male"]["Age"].mean()+train[train["Sex"]=="male"]["Age"].std(),train[train["Sex"]=="male"]["Age"].isna().sum()
)
female_age_train = np.random.randint(train[train["Sex"]=="female"]["Age"].mean()-train[train["Sex"]=="female"]["Age"].std(),train[train["Sex"]=="female"]["Age"].mean()+train[train["Sex"]=="female"]["Age"].std(),train[train["Sex"]=="female"]["Age"].isna().sum()
)

In [ ]:
train.loc[(train["Age"].isna()) & (train["Sex"]=="male"), "Age"] = male_age_train
train.loc[(train["Age"].isna()) & (train["Sex"]=="female"), "Age"] = female_age_train

In [ ]:
train.isnull().sum()

In [ ]:
male_age_test = np.random.randint(test[test["Sex"]=="male"]["Age"].mean()-test[test["Sex"]=="male"]["Age"].std(),test[test["Sex"]=="male"]["Age"].mean()+test[test["Sex"]=="male"]["Age"].std(),test[test["Sex"]=="male"]["Age"].isna().sum()
)
female_age_test = np.random.randint(test[test["Sex"]=="female"]["Age"].mean()-test[test["Sex"]=="female"]["Age"].std(),test[test["Sex"]=="female"]["Age"].mean()+test[test["Sex"]=="female"]["Age"].std(),test[test["Sex"]=="female"]["Age"].isna().sum()
)

In [ ]:
test.loc[(test["Age"].isna()) & (test["Sex"]=="male"), "Age"] = male_age_test
test.loc[(test["Age"].isna()) & (test["Sex"]=="female"), "Age"] = female_age_test

In [ ]:
test.isna().sum()

# EDA

In [ ]:
train

# Checking Impact of each Column on Output

## Pclass

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train.groupby(["Pclass"])["Survived"].mean()

## Gender

In [ ]:
train.groupby(["Sex"])["Survived"].mean()

## Embarked

In [ ]:
train.groupby(["Embarked"])["Survived"].mean()

In [ ]:
import seaborn as sns

## Age

In [ ]:
sns.distplot(train[train["Survived"]==1]["Age"],label="Survived")
sns.distplot(train[train["Survived"]==0]["Age"],label="Died")
plt.legend()
plt.show()


## Fare

In [ ]:
sns.distplot(train[train["Survived"]==1]["Fare"],label="Survived")
sns.distplot(train[train["Survived"]==0]["Fare"],label="Died")
plt.legend()

In [ ]:
train.drop(columns=["Ticket"], inplace = True)
test.drop(columns=["Ticket"], inplace = True)

## Feature Splitting

In [ ]:
train["Title"] = train["Name"].str.split(", ",expand = True)[1].str.split(". ",expand=True)[0]

In [ ]:
test["Title"] = test["Name"].str.split(", ",expand = True)[1].str.split(". ",expand=True)[0]

In [ ]:
test.head()

In [ ]:
train["ismarried"] = 0
test["ismarried"]=0

In [ ]:
train["ismarried"].loc[train["Title"] == "Mrs"]=1
test["ismarried"].loc[test["Title"] == "Mrs"]=1

In [ ]:
train.head()

In [ ]:
train["family"] = train["SibSp"]+train["Parch"]+1
test["family"] = test["SibSp"]+test["Parch"]+1

In [ ]:
train.head()

In [ ]:
plt.plot(train.groupby("family")["Survived"].mean())

In [ ]:
def cal(number):
    if number == 1:
        return "alone"
    elif number > 1 and number <5:
        return "medium"
    else:
        return "large"

In [ ]:
train["family_size"] = train["family"].apply(cal)

In [ ]:
test["family_size"] = test["family"].apply(cal)

In [ ]:
train.info()

In [ ]:
passenger_id = test["PassengerId"].values

In [ ]:
train.drop(columns=["SibSp","Parch","family","PassengerId","Name"], inplace = True)
test.drop(columns=["SibSp","Parch","family","PassengerId","Name"], inplace = True)

In [ ]:
train.groupby("ismarried")["Survived"].mean()

In [ ]:
train.drop(columns=["Title"], inplace = True)
test.drop(columns=["Title"], inplace = True)

In [ ]:
train.shape, test.shape

In [ ]:
train = pd.get_dummies(train, columns=["Pclass", "Sex", "Embarked", "family_size", "ismarried"], drop_first=True)
test = pd.get_dummies(test, columns=["Pclass", "Sex", "Embarked", "family_size", "ismarried"], drop_first=True)

In [ ]:
train.shape, test.shape

In [ ]:
train.head()

In [ ]:
x = train.iloc[:,1:].values
y = train.iloc[:,0].values

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
classifier = DecisionTreeClassifier()

In [ ]:
classifier.fit(x_train, y_train)

In [ ]:
y_pred = classifier.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
xf= test.iloc[:,:].values

In [ ]:
yf = classifier.predict(xf)

In [ ]:
final = pd.DataFrame()

In [ ]:
final["PassengerId"] = passenger_id
final["Survived"] = yf

In [ ]:
final

In [ ]:
final.to_csv("submissionf.csv",index = False)